# <center>Project 2</center>
### *Evgeniya Shevchuk*

### Models

In this project we recreated two models: SIRS and SIR.

Number of time steps - 30. Quantity of intially infected people - 50.

### Method

The method that was used here is simply vaccinate 10% of nodes with the highest node degree as far as other strategies did not work. I also tried using highest node degree method after some steps (5% at the beginning of epidemics, next 5% on the 4th step when we know who is already infected) but the results were worse than this method. If we vaccinate infected node, then it gets the immunity for the further steps after he gets susceptible again.

Despite we do 5 runs, the results are still random, so on further runs, results could be slightly different.

### Data

Use PLOS [dataset](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1001109#s2) of sexual escort market. It contains buyers as well as sellers of escort services. Contains about 15.000 nodes. As it is the real-world dataset, is it interesting to test a epidemic model on it. Use SIRS model- modification of SIR model, as 'recovering' nodes return to susceptible population after some time. Also, modify model to count recovery in days, not probabilistically.

In [1]:
from urllib.request import urlopen
import csv
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import random
from operator import itemgetter
from scipy.sparse import csr_matrix
import scipy.sparse.linalg as lg
from numpy.linalg import lstsq
import numpy as np
%matplotlib inline


In [2]:
dat = open('Dataset_S1.csv','rt')
reader = csv.reader(dat,delimiter=';')
Dset = list(reader)

In [3]:
g = nx.Graph(directed=False)
edgelist = []
for i in range(len(Dset)):
    if i>=24:
        edgelist.append((Dset[i][0],Dset[i][1]))
g.add_edges_from(edgelist)
g = list(nx.connected_component_subgraphs(g))[0]

Below example of use SIRS model - modification of SIR model, as our 'recovering' nodes return to susceptible population after some time.
We also modify model to count recovery in days, not probabilistically.

\begin{equation}
   \begin{cases}
   \cfrac{ds_i(t)}{dt} = -\beta s_i(t)\sum\limits_j A_{ij} x_j(t) + \gamma r_i(t) \\ 
   \cfrac{dx_i(t)}{dt} = \beta s_i(t)\sum\limits_j A_{ij} x_j(t) - \gamma x_i(t)\\
   \cfrac{dr_i(t)}{dt} = \gamma x_i(t) - \gamma r_i(t)
  \end{cases}
  \\
  x_i(t) + s_i(t) + r_i(t) = 1
\end{equation}

**Default parameters for Epidemic modelling**

In [4]:
#creating sorted adjacency matrix to iterate
#it is sorted by node degree-will be used later
nodelist = g.nodes()
deg = dict(nx.degree(g))
sorted_deg = sorted(deg.items(),key=itemgetter(1),reverse=True)
sortlist =[a[0]for a in sorted_deg]
A = np.array(nx.adjacency_matrix(g,nodelist = sortlist).todense())
n = len(nodelist) #total quantity

In [5]:
#defining randomiser for infection spread
#also choosing number of initially infected individuals
idx = np.random.choice(len(nodelist),50)
I = np.zeros(len(nodelist))
R = np.zeros(len(nodelist))
S = np.ones(len(nodelist))
I[idx]=1
S[idx]=0
def lottery(y,alph):
    V = np.zeros(len(y))
    for i in range(len(y)):
        x = random.random()
        if x < (1-(1-alph)**y[i]):
            V[i] = 1
        else:
            V[i] = 0
    return V

del nodelist, deg, sorted_deg,sortlist

In [6]:
#our iterative epidemic function, based on model
def epidemic_SIRS(bet,gam,delt,R,I,S,A,T):
    t=1
    rnd = list()
    R = np.zeros(len(I))
    M = np.zeros(len(I)) # for counting totally infected people
    di = I
    while t <=T:
        if t == 1:  # on the first step we found top 10% of nodes and vaccinate them
            V = np.zeros(len(I))
            vac = int(n*0.1)
            V[vac:] = 1   # for not vacinated
        D = csr_matrix(A)*csr_matrix(np.diag(S))
        ss_matr = (csr_matrix(I)*D).todense().tolist()[0]
        ds = lottery(R,delt)
        dr = lottery(I,gam)
        di = lottery(ss_matr,bet)*V  # here we multiply it to the vaccinated vector value by value
        dI = np.subtract(di,dr)
        dS = np.subtract(ds,di)
        dR = np.subtract(dr,ds)
        I = I+dI
        S = S+dS
        R = R+dR
        M = M+I
        rn = [sum(I),sum(S),sum(R)]
        rnd.append([round(x/len(I),2) for x in rn])
        t+=1
    total = n - (np.count_nonzero(M)) # counting how many nodes were never infected
    return(total)


**Three competition scenarios for model parameters**

In [7]:
s = [] # for each scenario we do 5 runs to average it somehow, however the computation takes quite much time
for i in range(5):
    T1 = epidemic_SIRS(0.3,0.1,0.1,R,I,S,A,30)
    s.append(T1)

In [8]:
s2 = []
for i in range(5):
    T2 = epidemic_SIRS(0.2,0.5,0.2,R,I,S,A,30)
    s2.append(T2)

In [9]:
s3 = []
for i in range(5):
    T3 = epidemic_SIRS(0.7,0.05,0.5,R,I,S,A,30)
    s3.append(T3)

In [10]:
t1 = np.mean(s)
t2 = np.mean(s2)
t3 = np.mean(s3)

Here, we compute and review different ways the infection can spread, given different parameters  

In [11]:
def epidemic_SIR(bet,gam,R,I,S,A,T):
    t=1
    rnd = list()
    R = np.zeros(len(I))
    M = np.zeros(len(I)) 
    di = I
    while t <=T:
        if t == 1:
            V = np.zeros(len(I))
            vac = int(n*0.1)
            V[vac:] = 1   
        D = csr_matrix(A)*csr_matrix(np.diag(S))
        ss_matr = (csr_matrix(I)*D).todense().tolist()[0]
        dr = lottery(I,gam)
        di = lottery(ss_matr,bet)*V
        dI = np.subtract(di,dr)
        dS = -di
        dR = dr
        I = I+dI
        S = S+dS
        R = R+dR
        M = M+I
        rn = [sum(I),sum(S),sum(R)]
        rnd.append([round(x/len(I),2) for x in rn])
        t+=1
    total = n - (np.count_nonzero(M))
    return(total)

In [12]:
s4 = []
for i in range(5):
    T4_h =(epidemic_SIR(0.2,0.1,R,I,S,A,3))
    s4.append(T4_h)
t4 = np.mean(s4)


In [13]:
s5 = []
for i in range(5):
    T4_h3 =(epidemic_SIR(0.2,0.1,R,I,S,A,5))
    s5.append(T4_h3)
t5 = np.mean(s5)

In [14]:
s6 = []
for i in range(5):
    T4_h2 =(epidemic_SIR(0.2,0.1,R,I,S,A,10))
    s6.append(T4_h2)
t6 = np.mean(s6)

In [15]:
av = (t1+t2+t3+t4+t5+t6)/6  #average value
av

14302.633333333331

In [16]:
import pandas as pd
list1 = ['1','2','3','4','5','6']
list2 = [t1,t2,t3,t4,t5,t6]
data = dict(col1=list1, col2=list2)
df = pd.DataFrame(data)
df.to_csv('results1.csv',header=['id','score'],index=False)

This method is not the most efficient since it takes quite a lot of time to compute. But still the results are pretty good and hard to beat with the small improvements that I have tried.